imports ...

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


***
##  10 Thousand German News Dataset - Topic Classification

Wir laden das 10k German News Dataset. Es enthält ca. 10k Online News-Artikel einer österreichischen Tageszeitung.  
Die Artikel wurden von (menschlichen) Annotierern in verschiedene thematische Klassen eingeteilt.  
Jeder Artikel ist genau einer der folgenden neun Kategorien zugeordnet:  

+ 'Etat'
+ 'Inland'
+ 'International'
+ 'Kultur'
+ 'Panorama'
+ 'Sport'
+ 'Web'
+ 'Wirtschaft'
+ 'Wissenschaft'

Mehr Details zum Datenset unter: https://tblock.github.io/10kGNAD/
  
Zunächst laden wir das Datenset in einen Dataframe "gnd"

In [ ]:
gnd = pd.read_csv('10kGNAD_train.csv', header=None, sep=';', quotechar="'", names=['label', 'text'])
pd.set_option('display.max_colwidth', -1)
print("Umfang des Datensets:",gnd.shape)
gnd.head(3)

Kurze Exploration der Daten

In [ ]:
print(gnd.label.value_counts(normalize=True).round(3))
_= gnd.label.value_counts().plot(kind="pie", )
plt.show()
_= gnd.label.value_counts().plot(kind="barh", )

Wir splitten wieder in Training (70%) und Test-Set (30%):

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(gnd, test_size=.3, stratify=gnd.label)
df_train.shape, df_test.shape

In [ ]:
df_train.label.value_counts(normalize=True).round(3)

Wir importieren die notwendigen Classifier & Tools aus Scikit-Learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 

Dann bauen wir eine "Pipeline", in der automatisch das Preprocessing vorgenommen wird.  
Auch dafür stellt Scikit-Learn mit der Klasse "Pipeline" ein sehr nützliches Werkzeug bereit.  

In [ ]:
# classifier pipeline mit Support Vektor Machine als Classifier definieren
lsvc_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('clf', LinearSVC(dual=False, C=1.6, class_weight="balanced")),
])

# classifier mit vorgeschalteter Pipeline trainieren:
lsvc_classifier.fit(df_train['text'], df_train['label'])  # train the classifier

# trainierten classifier auf die Testdaten anwenden, um den Accuracy Score zu ermitteln:
predicted = lsvc_classifier.predict(df_test['text'])  # predict the test set 
print(classification_report(predicted, df_test.label))

Zum Vergleich dasselbe nochmal mit einem anderen Classifier: "Logistische Regression"

In [ ]:
logReg_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),               
  ('clf', LogisticRegression(class_weight="balanced", solver='lbfgs', multi_class='auto'),),
])

logReg_classifier.fit(df_train['text'], df_train['label'])         # train the classifier
predicted = logReg_classifier.predict(df_test['text'])             # predict the test set 
acc = np.mean(predicted == df_test['label'])                       # calculate the accuracy
print(classification_report(predicted, df_test.label))

# Parkplatz

In [ ]:
# os.chdir("F://Data/Data Science Uni/40200/BMW Bank Seminar/Data Sets")